## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
nx = 9
ny = 6

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        print('Appended corners from ' + fname)
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
    else:
        print('Skipped ' + fname)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    

Skipped ./camera_cal\calibration1.jpg
Appended corners from ./camera_cal\calibration10.jpg
Appended corners from ./camera_cal\calibration11.jpg
Appended corners from ./camera_cal\calibration12.jpg
Appended corners from ./camera_cal\calibration13.jpg
Appended corners from ./camera_cal\calibration14.jpg
Appended corners from ./camera_cal\calibration15.jpg
Appended corners from ./camera_cal\calibration16.jpg
Appended corners from ./camera_cal\calibration17.jpg
Appended corners from ./camera_cal\calibration18.jpg
Appended corners from ./camera_cal\calibration19.jpg
Appended corners from ./camera_cal\calibration2.jpg
Appended corners from ./camera_cal\calibration20.jpg
Appended corners from ./camera_cal\calibration3.jpg
Skipped ./camera_cal\calibration4.jpg
Skipped ./camera_cal\calibration5.jpg
Appended corners from ./camera_cal\calibration6.jpg
Appended corners from ./camera_cal\calibration7.jpg
Appended corners from ./camera_cal\calibration8.jpg
Appended corners from ./camera_cal\calibrat

In [4]:
import os

def CalSingle(img, save = False):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        if save:
            cv2.imwrite('output_images\\corners.jpg', img)
        cv2.imshow('img', img)
        cv2.waitKey(500)
        cv2.destroyAllWindows()
    else:
        print('Skipped ')
    return

# Test Function
CalSingle(cv2.imread(images[11]), True)

## Compute camera matrix and correct distortion

In [32]:
def undistort(img, mtx, dist, save = False):
    # Use cv2.undistort()
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    if save:
        cv2.imwrite('output_images\\undist.jpg', undist)
    return undist

# Test Function

undistorted = undistort(cv2.imread(images[11]), mtx, dist, True)

## Inspect test images

In [31]:
a = undistort(cv2.imread('test_images/straight_lines1.jpg'), mtx, dist, True)
plt.imshow(a)
cv2.imwrite('output_images\\undist_straight_lines1.jpg', a)
#src = np.float32([[567,446],[706,446],[133,663],[1171,663]])
#dst = np.float32([[100,100],[1200,100],[100,900],[1200,900]])x = int(a.shape[1])
x = int(a.shape[1])
y = int(a.shape[0])
src = np.float32([[x/2-70,446],[x/2+70,446],[x/2-550,650],[x/2+550,650]])
dst = np.float32([[40,0],[x-40,0],[80,700],[x-80,700]])
print('x = {}, y = {}'.format(x,y))
poly = np.array([[(750,446),(530,446),(90,663),(1190,663)]])
print(poly)
print(src)
print(dst)

x = 1280, y = 720
[[[ 750  446]
  [ 530  446]
  [  90  663]
  [1190  663]]]
[[  570.   446.]
 [  710.   446.]
 [   90.   650.]
 [ 1190.   650.]]
[[   40.     0.]
 [ 1240.     0.]
 [   80.   700.]
 [ 1200.   700.]]


## Apply Sobel thresholding in x or y

In [7]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255, convert = True):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    if convert:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    if orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    sbinary = np.zeros_like(scaled_sobel)
    # is > thresh_min and < thresh_max
    sbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    # 6) Return this mask as your binary_output image
    return sbinary


## Magnitude of the gradient

In [8]:
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    abs_sobel = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a binary mask where mag thresholds are met
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return sbinary


## Direction of the gradient

In [9]:
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    abs_sobel = np.sqrt(sobelx**2 + sobely**2)
    dir_sobel = np.arctan2(sobely, sobelx)
    scaled_abs_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a binary mask where direction thresholds are met
    sbinary = np.zeros_like(dir_sobel)
    sbinary[(dir_sobel >= thresh[0]) & (dir_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return sbinary


## HLS select

In [10]:
def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    test = hls[:,:,2]
    # 2) Apply a threshold to the S channel
    binary_output = np.zeros_like(test)
    binary_output[(test > thresh[0]) & (test <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output


## Perspective Transform

In [11]:
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

def corners_unwarp(img, M = M): 
    
    img_size = (img.shape[1], img.shape[0])
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped

## Helper functions

In [12]:
import math

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


## Line Tracking

In [13]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
left_line = Line.Line()
right_line = Line.Line()

## Pipeline

In [27]:
test_list = os.listdir("test_images/")

def pipeline(image_name, write_out = False):
    
    if write_out:
        image = mpimg.imread("test_images/" + image_name)
    else:
        image = image_name
        
    img_original = np.copy(image)
    
    undist = undistort(image, mtx, dist)
    sobel_x = abs_sobel_thresh(undist, orient='x', thresh_min=50, thresh_max=255)
    sobel_mag = mag_thresh(undist, sobel_kernel=3, mag_thresh=(30, 100))
    sobel_dir = dir_threshold(undist, sobel_kernel=15, thresh=(0.8, 1.2))
    hls_s = abs_sobel_thresh(hls_select(undist, thresh=(90, 255)), orient='x', thresh_min=50, thresh_max=255, convert = False)
    
    #test = np.zeros_like(sobel_x)+ 255
    #test = np.dstack((test,test,test))
    # color binary
    #color_binary = np.zeros_like(sobel_x)
    sobel_binary = np.dstack(( sobel_x, sobel_mag, sobel_dir ))

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sobel_x)
    combined_binary[(sobel_x == 1) | (sobel_mag == 1) & (sobel_dir == 1) | (hls_s == 1)] = 1 # (sobel_mag == 1) & (sobel_dir == 1)
    combined_binary = region_of_interest(combined_binary, poly)
    result = np.dstack(( combined_binary*255, combined_binary*255, combined_binary*255 ))
    
    binary_warped = corners_unwarp(combined_binary)
    
    histogramtest = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    lane_width = rightx_current-leftx_current
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    leftx_var = 0
    rightx_var = 0

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_var = np.var(nonzerox[good_left_inds])
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_var = np.var(nonzerox[good_right_inds])
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
        if (leftx_var > 0) & (rightx_var > 0):
            if np.absolute(rightx_current-leftx_current-700)>40:
                if rightx_var>leftx_var:
                    rightx_current = leftx_current+700
                else:
                    leftx_current = rightx_current-700
            
        leftx_var = 0
        rightx_var = 0

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    y_eval = np.max(ploty)
    #left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    #right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    window_img = np.zeros_like(out_img)
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    margin = 20
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    #result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    #plt.imshow(result)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    window_img = corners_unwarp(window_img, Minv)
    overlay = weighted_img(window_img, undist)
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,50)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2
    cv2.putText(overlay,"Radius of Curvature: " + 
                str(np.mean([left_curverad, right_curverad])), 
                bottomLeftCornerOfText, 
                font, 
                fontScale,
                fontColor,
                lineType)
    
    if write_out:
        print(left_curverad, 'm', right_curverad, 'm')
        print("Radius of Curvature: " + 
                str(np.mean([left_curverad, right_curverad])))
        if 0:
            # Plotting thresholded images
            f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(20,10))
            ax1.set_title('Stacked sobel thresholds')
            ax1.imshow(corners_unwarp(sobel_binary))

            ax2.set_title('HLS S channel threshold')
            ax2.imshow(corners_unwarp(np.dstack((hls_s, hls_s, hls_s))*255))

            ax3.set_title('Combined S channel and gradient thresholds')
            ax3.imshow(corners_unwarp(combined_binary), cmap='gray')

            ax4.set_title('Overlayed Original')
            ax4.imshow(overlay) #region_of_interest(img_original,poly))

            ax5.set_title('Histogram')
            ax5.plot(histogramtest)

            ax6.set_title('Lines?')
            ax6.imshow(out_img)
            ax6.plot(left_fitx, ploty, color='yellow')
            ax6.plot(right_fitx, ploty, color='yellow')
            #ax6.xlim(0, 1280)
            #ax6.ylim(720, 0)

        # Save images
        cv2sillyness = cv2.cvtColor(overlay,cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join('output_images',image_name + '_final.jpg'), cv2sillyness)
        
    return overlay

[pipeline(x,True) for x in test_list];

15256.3773263 m 6301.94801519 m
Radius of Curvature: 10779.1626708
11466.3844436 m 8559.48088093 m
Radius of Curvature: 10012.9326623
2211.2503627 m 1832.28783286 m
Radius of Curvature: 2021.76909778
1847.77062653 m 2201.19875536 m
Radius of Curvature: 2024.48469095
2010.30850957 m 2044.88951127 m
Radius of Curvature: 2027.59901042
1607.41929762 m 1872.14131407 m
Radius of Curvature: 1739.78030585
2835.19509297 m 1785.17379166 m
Radius of Curvature: 2310.18444232
4363.54324675 m 3688.4044283 m
Radius of Curvature: 4025.97383753


## Video

In [28]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = pipeline(image)

    return result

first_video = 'project_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip('project_video.mp4') #.subclip(0,5)
##clip1 = VideoFileClip('project_video.mp4')
first_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!

%time first_clip.write_videofile(first_video, audio=False)

[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [08:32<00:00,  2.53it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

Wall time: 8min 34s


In [29]:
second_video = 'challenge_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip2 = VideoFileClip('challenge_video.mp4') #.subclip(0,5)
##clip2 = VideoFileClip('challenge_video.mp4')
second_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!

%time second_clip.write_videofile(second_video, audio=False)

[MoviePy] >>>> Building video challenge_video_out.mp4
[MoviePy] Writing video challenge_video_out.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 485/485 [03:11<00:00,  2.25it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_out.mp4 

Wall time: 3min 13s
